In [5]:
import os
import pandas as pd
import random
import shutil
import librosa
import numpy as np
import soundfile as sf

/home/bongseok/anaconda3/envs/vad/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/bongseok/anaconda3/envs/vad/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [6]:
# Freesound dataset
# negative data로 삼을 소리의 유형을 고릅니다.
# 여기서는 가정에서 날 법한 소리 위주로 선정했습니다.

OtherSound_class = ['Cat', 'Bell', 'Applause', 'Bark', 'Computer_keyboard', 'Clock', 'Cellphone_buzz_and_vibrating_alert',
              'Drip','Pour', 'Cough', 'Dog', 'Domestic_animals_and_pets','Piano', 'Air_conditioning',
              'Domestic_sounds_and_home_sounds', 'Hair_dryer', 'Meow', 'Vacuum_cleaner','Coin_(dropping)',
              'Whimper_(dog)', 'Yip', 'Purr', 'Electric_toothbrush', 'Printer', 'Steam', 'Snoring', 'Pig',
              'Spray', 'Water', 'Microwave_oven', 'Cat_communication', 'Growling', 'Typing',
               'Mechanisms', 'Camera', 'Boiling', 'Hands', 'Sink_(filling_or_washing)', 'Scissors',
              'Hands', 'Knock', 'Writing', 'Walk_and_footsteps', 'Toilet_flush', 'Door', 'Spray', 'Clicking',
              'Wood', 'Sliding_door', 'Engine', 'Rain', 'Mosquito', 'Toothbrush','Tearing', 'Sink_(filling_or_washing)',
               'Sneeze', 'Tap', 'Mechanical_fan', 'Ding-dong', 'Gargling',
              'Door', 'Telephone_bell_ringing', 'Guitar', 'Dishes_and_pots_and_pans', 'Breathing',
              'Doorbell', 'Knock', 'Alarm', 'Finger_snapping', 'Zipper_(clothing)', 'Keys_jangling',
              'Blender', 'Glass', 'Clapping', 'Wind_noise_(microphone)', 'Wind', 'Yawn', 'Biting', 'Clip-clop',
               'Drum', 'Gurgling', 'Packing_tape_and_duct_tape', 'Snap', 'Sniff', 'Snare_drum','Tambourine','Tick',
               'Tick-tock', 'Typewriter', 'Squawk', 'Liquid', 'Heart_sounds_and_heartbeat', 'Fart', 'Drum_kit',
               'Digestive', 'Chopping_(food)', 'Bathtub_(filling_or_washing)', 'Bass_drum','Alarm_clock',
               'Acoustic_guitar', 'Chop', 'Drum_roll', 'Toot', 'Throat_clearing', 'Squeak', 'Screech',
               'Scratching_(performance_technique)', 'Insect', 'Hiss', 'Hammer', 'Fly_and_housefly',
               'Cupboard_open_or_close', 'Buzz', 'Bird', 'Bus', 'Car', 'Crumpling_and_crinkling',
               'Car_alarm', 'Cash_register', 'Bass_guitar', 'Gong','Light_engine_(high_frequency)',
               'Tools', 'Wind_chime', 'Vehicle', 'Velcro_and_hook_and_loop_fastener', 'Stomach_rumble', 'Gears',
               'Truck', 'Stream', 'Ringtone','Gargling', 'Chink_and_clink', 'Electric_guitar',
               'Ukulele','Trickle_and_dribble','Strum','Stir','Squish','Skidding','Power_tool','Musical_instrument',
               'Jackhammer','Keyboard_(musical)','Harmonica','Frying_(food)','Busy_signal',
               'Bird_vocalization_and_bird_call_and_bird_song','Accelerating_and_revving_and_vroom', 'Babbling',
                'Bicycle', 'Bicycle_bell', 'Bird_flight_and_flapping_wings', 'Bleat','Caterwaul','Caw', 'Cello',
               'Chainsaw','Choir','Clarinet','Cluck','Clunk',"Dental_drill_and_dentist's_drill",'Electric_shaver_and_electric_razor',
               'Helicopter','Jingle_bell','Moo','Organ','Trombone','Violin_and_fiddle','Water_tap_and_faucet',
               'Violin_and_fiddle', 'Tire_squeal','Sawing','Rodents_and_rats_and_mice','Raindrop',
               'Percussion','Owl','Sewing_machine','Synthesizer','Synthetic_singing','Train_whistle','Waves_and_surf',
               'Waterfall','Timpani','Vehicle_horn_and_car_horn_and_honking','Steam_whistle','Skateboard',
               'Saxophone','Rhodes_piano','Race_car_and_auto_racing','Pulleys','Plucked_string_instrument','Police_car_(siren)',
               'Oboe','Motorcycle','Howl', 'Howl_(wind)','Frog','Fire_alarm', 'Drawer_open_or_close', 'Drill', 'Duck', 
               'Electric_piano', 'Emergency_vehicle', 'Coo', 'Cowbell', 'Crack', 'Crackle', 'Crash_cymbal', 'Cricket',
               'Accordion', 'Air_horn_and_truck_horn','Alto_saxophone','Ambulance_(siren)','Bee_and_wasp_and_etc.',
               'Boat_and_Water_vehicle','Croak','Crow','Cymbal','Filing_(rasp)','Fill_(with_liquid)','Flute', 'Glockenspiel',
               'Goose', 'Grunt','Hi-hat','Motor_vehicle_(road)','Rustling_leaves','Sheep','Splash_and_splatter',
               'Speech_synthesizer', 'Thump_and_thud', 'Thunder', 'Thunderstorm', 'Thunk', 'Traffic_noise_and_roadway_noise',
               'Train','Tubular_bells','Telephone_dialing_and_DTMF','Tabla','Siren','Shuffling_cards',
               'Rail_transport', 'Rain_on_surface','Marimba_and_xylophone','Hoot',
                'Dial_tone', 'Didgeridoo', 'Double_bass', 'Engine_starting', 'Boom', 'Bowed_string_instrument', 'Brass_instrument',
                'Burping_and_eructation', 'Burst_and_pop', 'Buzzer', 'Aircraft', 'Artillery_fire','Car_passing_by',
               'Church_bell','Cutlery_and_silverware', 'Explosion','Fire', 'Fire_engine_and_fire_truck_(siren)',
               'Firecracker', 'Fireworks', 'Fixed-wing_aircraft_and_airplane','Harp', 'Harpsichord','Honk', 'Horse',
               'Idling','Oink', 'Orchestra','Pant', 'Pigeon_and_dove', 'Pizzicato','Natural_sounds', 'Music',
               'Ship','Shatter', 'Rimshot', 'Run','Slosh', 'Soprano_saxophone', 'Sitar',
               'Splinter', 'Steel_guitar_and_slide_guitar', 'Subway_and_metro_and_underground', 'Telephone',
               'Theremin','Wail_and_moan', 'Trumpet','Roar', 'Sanding', 'Mallet_percussion', 'Mantra', 'Maraca',
               'Neigh_and_whinny','Foghorn', 'French_horn','Chime','Chirp_and_tweet','Crushing','Slam',
                'Wild_animals', 'Wildfire', 'Wind_instrument_and_woodwind_instrument', 'Wobble','Train_horn',
               'Ratchet_and_pawl','Quack','Ocean','Gull_and_seagull','Lawn_mower','Chicken_and_rooster',
               'Chewing_and_mastication','Cattle_and_bovinae','Bassoon', 'Whistle', 'Whistling',
               'Machine_gun','Gunshot_and_gunfire','Cap_gun'
              ]

In [8]:
OtherSound_class = sorted(set(OtherSound_class))

In [9]:
len(OtherSound_class)

318

In [4]:
# print(OtherSound_class)

In [13]:
vocab = pd.read_csv(r'../Dataset_audio/Freesound_dataset/FSD50K.metadata/FSD50K.metadata/collection/vocabulary_collection_eval.csv')
collection = pd.read_csv(r'../Dataset_audio/Freesound_dataset/FSD50K.metadata/FSD50K.metadata/collection/collection_eval.csv')
OtherSound_audio_files_path = r'../Dataset_audio/Freesound_dataset/FSD50k.eval_audio/FSD50K.eval_audio'

In [14]:
vocab.head()

,filename,class,mids
0,0,Accelerating_and_revving_and_vroom,/m/07q2z82
1,1,Accordion,/m/0mkg
2,2,Acoustic_guitar,/m/042v_gx
3,3,Air_conditioning,/m/025wky1
4,4,Air_horn_and_truck_horn,/m/05x_td


In [174]:
#vocab.info()

In [175]:
#collection.head()

In [176]:
#collection.info()

In [15]:
collection['labels'] = collection['labels'].apply(lambda x: x.split(','))

In [179]:
#collection[:30]

In [16]:
# OtherSound_class 에 없는 소리 유형이 포함된 파일은 제외합니다.
# (예를 들어 기타 소리와 사람 목소리가 섞인 파일은 OtherSound로 학습되면 안 되기 때문입니다.)

erase_list = []
erase_idx = []
idx=0
for labels in collection['labels']:
    for label in labels:
        if label not in OtherSound_class:
            erase_list.append(label)
            erase_idx.append(idx)
    idx +=1

In [17]:
# 중복 제거
set(erase_list)

{'Baby_cry_and_infant_cry',
 'Baby_laughter',
 'Belly_laugh',
 'Booing',
 'Chant',
 'Chatter',
 'Cheering',
 'Child_singing',
 'Child_speech_and_kid_speaking',
 'Children_playing',
 'Children_shouting',
 'Chuckle_and_chortle',
 'Conversation',
 'Crowd',
 'Crowing_and_cock-a-doodle-doo',
 'Crying_and_sobbing',
 'Female_singing',
 'Female_speech_and_woman_speaking',
 'Fusillade',
 'Gasp',
 'Giggle',
 'Hubbub_and_speech_noise_and_speech_babble',
 'Human_sounds',
 'Human_voice',
 'Laughter',
 'Male_singing',
 'Male_speech_and_man_speaking',
 'Narration_and_monologue',
 'Rapping',
 'Rattle',
 'Rattle_(instrument)',
 'Respiratory_sounds',
 'Screaming',
 'Shout',
 'Sigh',
 'Singing',
 'Singing_bowl',
 'Speech',
 'Turkey',
 'Vibraphone',
 'Whispering',
 'Whoop',
 'Whoosh_and_swoosh_and_swish',
 'Yell'}

In [18]:
len(erase_list)

3046

In [19]:
c2 = collection.copy()
c2 = c2.drop(erase_idx)

In [20]:
c2

,fname,labels,mids
0,37199,[Electric_guitar],/m/02sgy
1,175151,[Electric_guitar],/m/02sgy
2,253463,[Electric_guitar],/m/02sgy
3,329838,[Electric_guitar],/m/02sgy
4,1277,[Electric_guitar],/m/02sgy
...,...,...,...
10226,266616,[Bleat],/m/07q0h5t
10227,137745,[Honk],/m/07qwf61
10228,418509,[Quack],/m/07qdb04
10229,76133,[Honk],/m/07qwf61


In [21]:
# 파일명이 있는 인덱스 찾기
#c2.index[c2['fname'] == 37199].tolist()[0]

def return_idx(dataframe, text):
    fname_idx = dataframe.index[dataframe['fname'] == text].tolist()
    return fname_idx[0]


In [22]:
filenames = c2['fname'].tolist()
labels = c2['labels'].tolist()

In [23]:
len(filenames)

8030

In [24]:
len(labels)

8030

In [25]:
type(filenames[1])

int

In [28]:
os.makedirs(r'../Dataset_audio/OtherSound', exist_ok=True)

selected_OtherSound_path = r'../Dataset_audio/OtherSound'
OtherSound_audio_files_path = r'../Dataset_audio/Freesound_dataset/FSD50k.eval_audio/FSD50K.eval_audio'

In [29]:
# 오디오 파일의 길이를 구하는 함수를 선업합니다.
def get_duration(path, fname):
    data, sr = librosa.load(os.path.join(path, fname))
    duration = librosa.get_duration(y=data, sr=sr)
    return duration

In [30]:
idx=0
labels = []

for file in os.listdir(OtherSound_audio_files_path):
    filename = int(os.path.splitext(file)[0])

    if filename in filenames:
        if get_duration(OtherSound_audio_files_path, file) <30: # 길이가 30초 미만인 파일만 선정합니다.
            fname_idx = return_idx(c2, int(filename))
            label = c2['labels'][int(fname_idx)][0]
            print(filename, label)
            labels.append(label)
            shutil.copy(
                        os.path.join(OtherSound_audio_files_path, file),
                        os.path.join(selected_OtherSound_path, label+'_'+str(filename)+'.wav')
            )
    else:
        print('this file was not selected')
        idx +=1

Gull_and_seagull
162312
Crash_cymbal
262265
Drill
164771
Harp
161686
this file was not selected
185951
Bark
382735
Gunshot_and_gunfire
167167
Wood
412186
Boiling
120833
this file was not selected
246057
Typewriter
273227
Acoustic_guitar
274499
Frying_(food)
62459
Wind_noise_(microphone)
165431
Frying_(food)
141531
Car
318973
Gunshot_and_gunfire
99
this file was not selected
365853
Fire
155462
Gong
419270
Accordion
45604
this file was not selected
393763
Engine
63885
Electric_guitar
319407
Snare_drum
362503
this file was not selected
194944
Walk_and_footsteps
74732
Purr
254878
Stream
203820
Double_bass
418181
Caw
330564
this file was not selected
262947
Oink
91559
this file was not selected
382589
this file was not selected
269260
Train
110103
Boiling
405210
this file was not selected
26184
Car
320066
this file was not selected
254081
Wind_noise_(microphone)
58969
Acoustic_guitar
36774
Snare_drum
217740
this file was not selected
130083
Trombone
334990
this file was not selected
235558


In [32]:
len(os.listdir(selected_OtherSound_path))

8015

In [31]:
# save labels of selected files
c2.to_csv("Labels_of_selected_noise.csv")